In [1]:
ytm_tenors = [3, 6, 12, 24, 36, 48, 60]
ytm_rates = [0.00138497, 0.00122494, 0.0012993, 0.00165278, 0.0021181, 0.00292553, 0.00397573]
currency = 'KRW'

In [2]:
coupon_tenor = 0    # CPNT in R
start_idx = 0       # STndx in R

if currency == 'USD':
    coupon_tenor = 2   # Semi-Annual
    start_idx = 1      # R에선 인덱스가 1부터 시작하므로 2이지만 파이썬은 0부터 시작
else:
    coupon_tenor = 4
    start_idx = 0      # R에선 인덱스가 1부터 시작하므로 1이지만 파이썬은 0부터 시작



# 지급 주기(월 환산)
ir_freq = 12 // coupon_tenor                        # QTSA in R 
nterm = len(ytm_tenors)
max_tenor = ytm_tenors[-1]                          # MaxT in R 
n_generated_tenor = divmod(max_tenor, ir_freq)[0]   # MaxN in R 


# 저장할 스팟커브 리스트
spot_rate = [0 for i in range(nterm)]

In [3]:
# 4개의 리스트가 모두 같은 크기로 초기화됨              YTMSPOT in R
ytm_spot_month = [(i+1) * ir_freq for i in range(n_generated_tenor)]      # 3개월 주기로 tenors를 만들어줌
ytm_spot_rate = [0 for i in range(n_generated_tenor)]
ytm_spot_df = [0 for i in range(n_generated_tenor)]
ytm_spot_annuity = [0 for i in range(n_generated_tenor)]

In [4]:
# R로직 그대로 소스
for i in range(start_idx, nterm, 1):
    ytm_spot_rate[divmod(ytm_tenors[i], ir_freq)[0]-1] = ytm_rates[i]

In [5]:
# R로직 그대로 소스를 변경한 소스
# 위에서 ytm_spot_month 와 ytm_tenors 와 비교 하여 같은 tenor가 있는 경우 그에 해당하는 이자율을 ytm_rates에서 가져와 ytm_spot_rate에 저장
for i, tenor in enumerate(ytm_tenors):
    if i < start_idx:
        continue
    else:
        idx = ytm_spot_month.index(tenor)
        if idx < 0:
            continue
        else:
            ytm_spot_rate[idx] = ytm_rates[i]

In [6]:
i = 0
while i < len(ytm_spot_rate):
    # 빈 rate(0의 값을 가짐)에 대해서 보간법으로 값을 채워주는 작업
    if ytm_spot_rate[i] == 0:
        
        zero_indecs = 1
        # i번째 뒤로 몇개나 rate가 0인지 확인
        while ytm_spot_rate[i + zero_indecs] == 0:
            zero_indecs += 1
        
        # 보간법 이용해서 0인 부분 채워넣기
        for j in range(zero_indecs):
            ytm_spot_rate[i + j] = (zero_indecs - j)/(zero_indecs+1) * ytm_spot_rate[i-1] + (j+1 )/(zero_indecs+1) * ytm_spot_rate[i + zero_indecs]
        
        i += zero_indecs
    i+=1

In [7]:
for i in range(2):
    ytm_spot_df[i] = 1/(1 + ytm_spot_month[i]/12 * ytm_spot_rate[i])
    

In [8]:
sum = 0
for i in range(2):
    sum += ytm_spot_df[i]
    ytm_spot_annuity[i] = sum

In [9]:
for i in range(2, n_generated_tenor):
    ytm_spot_df[i] = (1- ytm_spot_rate[i] * ytm_spot_annuity[i-1] / coupon_tenor) / (1 + ytm_spot_rate[i] / coupon_tenor)
    ytm_spot_annuity[i] = ytm_spot_annuity[i-1] + ytm_spot_df[i]

In [10]:
for i in range(nterm):
    selected_tenor_idx = divmod(ytm_tenors[i], ir_freq)[0]

    if i == 0:
        spot_rate[i] = ytm_rates[0]
    else:
        spot_rate[i] = (pow(ytm_spot_df[selected_tenor_idx - 1], (-1/selected_tenor_idx)) - 1) * coupon_tenor

In [11]:
spot_rate

[0.00138497,
 0.0012247524976647384,
 0.0012993142095538701,
 0.0016531270960671307,
 0.0021194358903890986,
 0.002930568479521334,
 0.0039897772998855885]